# Facebook comments Sentiment analysis

The aim of this notebook is to train and test a Neural Network to detect, if a Facebook comment is either positive or negative in nature, based on a sample of Facebook comments with attatched sentiment ratings.

## Loading packages

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fb = pd.read_csv('../input/fb_sentiment.csv')

## Exploring Data

The data that we have chosen are Facebook-comments (`FBPost`) , with sentiments (`Label`) rated *positive*, *negative* or *other*. There are rows in the dataset, with an uneven distribution of sentiments.

In this section, some minor cleaning will take place.

In [ ]:
fb.head()

In [ ]:
# lower-casing the coloumn names
fb.columns = map(str.lower, fb.columns)

In [ ]:
# checkin the shape of the DF
fb.shape

## Data preperation

### Preparing the Facebook comments

In [ ]:
#lowercasing the text and removing symbols though RegEx
import re
fb['fbpost'] = fb['fbpost'].apply(lambda x: x.lower())
fb['fbpost'] = fb['fbpost'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

With this Neural network, we want to predict, wether a comment is Positive (`P`) or Negative (`N`), so the comments with the sentiment labeled Other (`O`) is of no use to us, so it's removed from the dataset.

In [ ]:
fb = fb[fb.label != "O"]

Now to tokenize the actual Facebook comments:

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(fb['fbpost'].values)
X = tokenizer.texts_to_sequences(fb['fbpost'].values)
X = pad_sequences(X)

### Preparing the labels

Here the lables are checked after the removel of the "other" sentimented comments. Also some preperation to the algorithm, as preparing the test-, and training sets are done.

In [ ]:
fb.label.value_counts()

In [ ]:
Y = pd.get_dummies(fb['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

## The Neural Network

In this section, the algorithm is prepared with following features:

- The model is Sequential
- The model type is an LSTM model

In [ ]:
embed_dim = 200
lstm_out = 200

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
# Here we train the model
batch_size = 32
hist = model.fit(X_train, Y_train, epochs = 7, batch_size=batch_size, verbose = 2)

In [ ]:
#Plotting a histogram over the 7 epocs and plotting the accuracy and loss
history = pd.DataFrame(hist.history)
plt.figure(figsize=(7,7));
plt.plot(history["loss"]);
plt.plot(history["acc"]);
plt.title("Loss and accuracy of model");
plt.show();

In [ ]:
#Testing the model, and retrieveing score and accuracy:
score,acc = model.evaluate(X_test,Y_test)
print("score: %.2f" % (score))
print("accuracy: %.2f" % (acc))


In [ ]:
#now we validate for the models accuracy in predicting either a positive, or a negative score:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
x_test = X_test[:-validation_size]
y_test = Y_test[:-validation_size]

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,x_test.shape[1]),verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1
print("positive_acc", pos_correct/pos_cnt*100, "%")
print("negative_acc", neg_correct/neg_cnt*100, "%")

In [ ]:
#now testing  on a random sample from the Facebook comments on Kindle's page:
cmnt = ['your customer service is the absolute worst i now have a mess of books on my kindle']
#vectorizing the comment
cmnt = tokenizer.texts_to_sequences(cmnt)
cmnt = pad_sequences(cmnt, maxlen=203, dtype='int32', value=0)
print(cmnt)
sentiment = model.predict(cmnt,batch_size=2,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

What went wrong? 

Our sample size for the training-, and test set was in fact quite small (under 1000 comments), and the proportion of negative to positive comments was skewed about 1 to 9. So the algorithm was not really optimized on the basis of our data. This is the reason for the algorithm choosing the wrong sentiment in the sample-test above.